# A FAIRE dans CNN_SL :
* chargement d'un modèle
* chargement des poids d'un modèle
* fonction eval_state

In [ ]:
import keras
import go 
import CNN_policy

In [ ]:
import numpy as np
# definition d'un joueur

class Player(object):

    def __init__(self, policy_function):
        self.policy = policy_function

    def get_move(self, state):
        # list with sensible moves
        sensible_moves = [move for move in state.get_legal_moves(include_eyes=False)]
        
      
        
        #parcourt les coups possible et ressort celui qu'aurai jouer le policy network    
        if len(sensible_moves) > 0:
            a=np.random.randint(0,len(sensible_moves),1) #on prend un coup au hasard
            return(sensible_moves[a[0]])
        
        """"
            move_probs = self.policy.eval_state(state, sensible_moves)  # A AJOUTER 
            max_prob = max(move_probs, key=itemgetter(1))
            return max_prob[0]
        """
        # No 'sensible' moves available, so do pass move
        return go.PASS_MOVE


In [ ]:
# a mettre dans la classe 

## Chargement du modèle SL

* A FAIRE ajouter à CNN_SL de quoi charger un modele et ses poids

In [ ]:
filename="test.json"
policy_pl=CNN_policy.CNN()
policy_pl.load () 

In [ ]:
initial_weights=""
policy_pl.model.load_weights(args.initial_weights)
policy_op=CNN_policy.CNN()
policy_op = CNN_SL.load (file_name)  
policy_op.model.load_weights(args.initial_weights)

il faut compiler les modeles ?

In [ ]:
# pour l'apprentissage plus tard
from keras.optimizers import SGD
opti = SGD(lr=0.1)
player.policy.model.compile(optimizer=opti)

## Création des joueurs
-> la fonction get move renvoye le coup prédit par le SL en fct de l'état du jeu


In [ ]:
# a retirer quand le model sera chargé
from keras.models import  Model,Sequential
policy_pl=Sequential()
policy_op=Sequential()

In [ ]:
player = Player(policy_pl)
opponent =Player(policy_op)

## Déroulement des partie

In [ ]:
import go
import time

In [ ]:
#initialisation
nb_partie=1000
taille=9
import features as ft
conv=ft.Preprocess()

In [ ]:
# deroulement
start=time.time()
s1=0 #tps premiere partie

etat = [go.GameState(size=taille) for _ in range(nb_partie)] #liste des parties actuelle
coups=[[] for _ in range(nb_partie)]  #liste des coups joués
parties=[[] for _ in range(nb_partie)] # liste des etats du jeu
id_gagne=[] #liste des indices des parties gagnées
ratio=0

op_couleur="BLACK"
pl_couleur="WHite"
# on joue le premier coup de chaque partie
for i in range(nb_partie):
    coup=opponent.get_move(etat[i]) 
    coups[i].append(coup)
    etat[i].do_move(coup)
    parties[i].append(conv.state_to_tensor(etat[i]))

#on joue tout les coups
actuel=player
ancien=opponent
fin=0 # pour arreter la boucle
while(fin<nb_partie): 
    
    for i in range(nb_partie):
            if(etat[i].is_end_of_game==False): # vérifie que la partie n'est pas fini
                coup = actuel.get_move(etat[i]) # on récupere le coup jpué 
                coups[i].append(coup) # on le sauvegarde
                etat[i].do_move(coup) #on le joue
                parties[i].append(conv.state_to_tensor(etat[i])) #on sauvegarde l'état du jeu
                
                
                if(etat[i].is_end_of_game==True): 
                    fin+=1  # pour arreter la boucle
                    if(etat[i].get_winner()==-1): # -1 pour blanc
                        id_gagne.append(i)
                        ratio+=1
                        
                if (fin==1&s1==0):
                    print("1ere partie finie en %f secondes" % start-time.time())
                    s1=1
    #vis_gs(parties[1])
    

    #on change de joueur
    temp=actuel
    actuel=ancien
    ancien=actuel

ratio /=float(nb_partie)
print("%d executé en %f secondes"%(nb_partie,start-time.time()))
print("ratio de victoire: %f" % ratio)



In [ ]:
# verif
id_gagne

In [ ]:
parties[2]

## Entrainement

In [16]:
import keras.backend as K

for i in range(nb_partie):
    
    if i in id_gagne:
        # on apprend le modele
        opti.lr=K.abs(opti.lr) *1
        policy_pl.train_on_batch(parties[i],coups[i])  #a verifier
    else:
        #on dessaprend le modele --> donner des poids negatifs
        opti.lr=K.abs(opti.lr) *-1
        policy_pl.train_on_batch(parties[i],coups[i])  #a verifier